<h1 style="background-color: green;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐תהליך הסבת גבולות מפות הסדר לבנק"ל מודרני🌐

In [ ]:
print(' Blocks regulation borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
from configs import CNFG
import dask_geopandas as daskgpd

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('   Reading inputs')

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תהליכי קדסטר קיימת

In [ ]:
CadasterProcessBorder_cols = ['CADASTER_PROCESS', 'GUSHNUM', 'GUSHSUFFIX', 'GUSH_STATUS', 'geometry']

CadasterProcessBorder = daskgpd.read_file(CNFG.CADSDE + 'CadasterProcessBorder.gdb', layer = 'CadasterProcessBorder', columns = CadasterProcessBorder_cols, npartitions = 8).compute()

del CadasterProcessBorder_cols

<h1 style="text-align:right;font-size:100%">
שכבת גושי קדסטר רציפים

In [ ]:
SubGushAll_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'STATUS', 'geometry']

SubGushAll = daskgpd.read_file(CNFG.Arcstorm + 'SUB_GUSH_ALL.gdb', layer = 'SUB_GUSH_ALL', columns = SubGushAll_cols, npartitions = 8).compute()

del SubGushAll_cols

<h1 style="text-align:right;font-size:100%">
שכבת גושי מפתח כליונות

In [ ]:
SheetK_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'STATUS', 'geometry']

SheetK = daskgpd.read_file(CNFG.Arcstorm + 'SHEET_K.gdb', layer = 'SHEET_K', columns = SheetK_cols, npartitions = 8).compute()

del SheetK_cols

<h1 style="text-align:right;font-size:100%">
טבלת מספרי זיהוי של גושים

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'GlobalID', 'IsTax'])
Blocks = Blocks[Blocks['IsTax'] == 0].drop(columns = 'IsTax')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

<h1 style="text-align:right;font-size:125%">
:עיבוד נתוני גושי קדסטר ברצף ומפתח גליונות
<h1 style="text-align:right;font-size:100%">
הגדרת רשת קואורדינטות של ישראל חדשה
<h1 style="text-align:right;font-size:100%">
חיבור השכבות לשכבה אחת
<h1 style="text-align:right;font-size:100%">
סינון לפי סטאטוסים שרלוונטיים לתהליכי הסדר בלבד והסרת רשומות ללא גיאומטריה
<h1 style="text-align:right;font-size:100%">
יצרת שדה של שם גוש מלא

In [ ]:
print('   Cleaning data')

In [ ]:
SubGushAll.to_crs(2039, inplace= True)
SheetK.to_crs(2039, inplace= True)

SubGushAll = pd.concat([SubGushAll, SheetK], ignore_index = True).reset_index(drop = True)

regulation_status = [7, 10, 11, 13, 15]   # בהסדר, חלוקה, מוקדמת, ארעית, סופית

SubGushAll = SubGushAll[(SubGushAll['STATUS'].isin(regulation_status)) & (SubGushAll['geometry'].notna())]

del regulation_status


SubGushAll['GUSH_SUFFIX'] = SubGushAll['GUSH_SUFFIX'].fillna(0)
SubGushAll[['GUSH_NUM', 'GUSH_SUFFIX', 'STATUS']] = SubGushAll[['GUSH_NUM', 'GUSH_SUFFIX', 'STATUS']].astype(int)
SubGushAll['BlockName'] = SubGushAll['GUSH_NUM'].astype(str) + '/' + SubGushAll['GUSH_SUFFIX'].astype(str)

<h1 style="text-align:right;font-size:125%">
:עיבוד נתוני גבולות תהליכי קדסטר
<h1 style="text-align:right;font-size:100%">
סינון גבולות תהליכי קדסטר קיימים לפי גבולות הסדר גוש

In [ ]:
regulation_borders = CadasterProcessBorder[CadasterProcessBorder['CADASTER_PROCESS'] == 1]
regulation_borders = regulation_borders.drop(columns = 'CADASTER_PROCESS')

del CadasterProcessBorder

<h1 style="text-align:right;font-size:100%">
יצרת שדה של שם גוש מלא
<h1 style="text-align:right;font-size:100%">
סינון שכבת גושים ממוזגת לאלו שלא נמצאים שכשבת גבולות תהליכי הסדר    
<h1 style="text-align:right;font-size:100%">
שינוי שמות שדות של שכבת גושים ממוזגת כך שיתאימו לשכבת גבולות תהליכי קדסטר    
<h1 style="text-align:right;font-size:100%">
מיזוג של שכבת גושים עם שכבת גבולות תהליכי קדסטר, הסרת רשומות שאין להן גיאומטריה   
<h1 style="text-align:right;font-size:100%">
יצירת שדה של שם מפה

In [ ]:
regulation_borders['GUSHSUFFIX'] = regulation_borders['GUSHSUFFIX'].fillna(0)

regulation_borders[['GUSHNUM', 'GUSHSUFFIX']] = regulation_borders[['GUSHNUM', 'GUSHSUFFIX']].astype(int)
regulation_borders['BlockName'] = regulation_borders['GUSHNUM'].astype(str) + '/' + regulation_borders['GUSHSUFFIX'].astype(str)

SubGushAll = SubGushAll[~SubGushAll['BlockName'].isin(regulation_borders['BlockName'])]
SubGushAll.rename(columns = {'STATUS':'GUSH_STATUS', 'GUSH_NUM':'GUSHNUM', 'GUSH_SUFFIX':'GUSHSUFFIX'}, inplace = True)

regulation_borders = pd.concat([regulation_borders, SubGushAll], ignore_index = True)
regulation_borders = regulation_borders[regulation_borders['geometry'].notna()]


regulation_borders['ProcessName'] = regulation_borders['GUSHNUM'].astype(str) + '/' + regulation_borders['GUSHSUFFIX'].astype(str) + ' הסדר'
regulation_borders.drop_duplicates('ProcessName', keep = 'first', inplace = True)

<h1 style="text-align:right;font-size:100%">
יצירת שדה משותף של מספר גוש מלא ואיחוד עם שכבת גושי קדסטר מודרנית לקבלת מזהה גוש

In [ ]:
Blocks.rename(columns = {'GlobalID' : 'BlockUniqueID'}, inplace = True)
regulation_borders = regulation_borders.merge(Blocks, left_on = 'BlockName', right_on = 'Name', how = 'left')

del Blocks

In [ ]:
print('   Data conversion')

<h1 style="text-align:right;font-size:100%">
המרת ערכי סטאטוסים קיימים לפי ערכים חדשים בהתאם למיפוי עכים
<h1 style="text-align:right;font-size:100%">
ערכים ריקים ימולאו ב-0 (חלוקה)

In [ ]:
regulation_borders['GUSH_STATUS'] = regulation_borders['GUSH_STATUS'].astype(int)

status_clf = {0    : 90, # לא ידוע - חלוקה
              7    : 90, # שומא-בהסדר - חלוקה
              9    : 90, # רישום ראשון-חלוקה 
              10   : 90, # חלוקה-חלוקה
              11   : 91, # מוקדמת- מפה מוקדמת בהכנה
              13   : 95, # ארעית - מפה ארעית בהכנה
              15   : 99, # סופית - מפה סופית בהכנה
              17   : 90, # ירדני בהסדר - חלוקה
              None : 90} # ללא ערך- חלוקה


regulation_borders['GUSH_STATUS'] = regulation_borders['GUSH_STATUS'].fillna(0)
regulation_borders['GUSH_STATUS'] = regulation_borders['GUSH_STATUS'].map(status_clf)
regulation_borders['GUSH_STATUS'] = regulation_borders['GUSH_STATUS'].astype(int)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('   Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("geometry", gpd.array.GeometryDtype),       
                       #("GlobalID", str),
                        ("ProcessName", str),
                        ("ProcessType", int),
                        ("Status", int),
                        ("BlockUniqueID", str),
                        ("GeodeticNetwork", int),
                       #("SurveyorLicenseID", int),
                        ("DataSource", int),
                       #("PlanName", str),  
                       #("ORACLE_ID", int),
                       #("Shape_Length", float),
                       #("Shape_Area", float),
                        ])

BlocksRegulationBorders = gpd.GeoDataFrame(np.empty(0, dtype=new_columns),
                                           geometry = 'geometry',
                                           crs = 2039)

<h1 style="text-align:right;font-size:100%">
איכלוס שדות

In [ ]:
BlocksRegulationBorders['geometry'] = regulation_borders['geometry']

BlocksRegulationBorders['DataSource'] = 5 #CAD

#BlocksRegulationBorders['SurveyorLicenseID'] = 0   # לא קיים מידע

#BlocksRegulationBorders['PlanName'] = None   # לא קיים מידע

BlocksRegulationBorders['Status'] = regulation_borders['GUSH_STATUS'].astype(int)

BlocksRegulationBorders['GeodeticNetwork'] = 3   # רשת ישראל התקפה

BlocksRegulationBorders['ProcessName'] = regulation_borders['ProcessName']

BlocksRegulationBorders['ProcessType'] = 9   # הסדר מקרקעין

BlocksRegulationBorders['BlockUniqueID'] = regulation_borders['BlockUniqueID']

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾

In [ ]:
print('   Appending results to CadasterProcessBorders')

In [ ]:
BlocksRegulationBorders.to_file(CNFG.Results, layer = 'PT9', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'w')